<a href="https://colab.research.google.com/github/jooyelee/alzheimer-detection-AI-idea-contest/blob/main/jooye_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping


In [ ]:

init_notebook_mode(connected=True)
RANDOM_SEED = 123
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline   
import matplotlib.pyplot as plt 
import seaborn as sns  


#from sklearn.preprocessing import MinMaxScaler
#from tensorflow.keras.layers import Dense, Dropout

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/sample2.csv")

In [ ]:
data.head()

In [ ]:
data.isnull().sum(1) # row 단위로 결측값 구하기

In [ ]:
data.describe()

In [ ]:
data.columns.get_loc("CERAD_J1")

In [ ]:
data.columns.get_loc("CDR")

In [ ]:
data.columns.get_loc("MMSE 총점")

In [ ]:
data.columns.get_loc("Left-Lateral-Ventricle")

# 데이터 분할

In [ ]:
cdr_df = data.iloc[:,33:36]
cdr1 = data.iloc[:,4]

cerad_col = data.columns[19:33]
cerad_df = data.iloc[:,19:33]

mri_col = data.columns[72:]
mri_df = data.iloc[:,72:]

cerad_cdr = pd.merge(cerad_df,cdr_df,left_index=True, right_index=True, how='left')

cerad_df=cerad_df.drop(columns=["CERAD_total1","CERAD_total2"])

In [ ]:
def draw_histograms(df, variables, n_rows):
    fig=plt.figure(figsize=(15,30))
   
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,2,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name)
    fig.tight_layout()  # Improves appearance a bit.
    plt.show()

draw_histograms(cerad_df, cerad_df.columns,15)

In [ ]:
cerad_df.head()

In [ ]:

plt.figure(figsize=(15,15))
sns.heatmap(data = cerad_df.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data = cerad_cdr.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
no_cerad_cdr = cerad_cdr.dropna()
nona = cerad_df.dropna()

In [ ]:
nona.tail(10)

# PCA & K MEANS CLUSTERING

In [ ]:
from sklearn.preprocessing import scale, normalize
scale = scale(nona)
normal = normalize(nona)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

scaled_pca = pca.fit_transform(scale)

print("pca.explained_variance_ratio_:",pca.explained_variance_ratio_)

cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.80) + 1
print('선택할 차원 수 :', d)


In [ ]:
pca = PCA()
normal_pca = pca.fit_transform(normal)

print("pca.explained_variance_ratio_:",pca.explained_variance_ratio_)

cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
print('선택할 차원 수 :', d)

In [ ]:
normal_pca_df = pd.DataFrame(normal_pca)

In [ ]:
normal_pca_df.head()

In [ ]:
normal_pca_cdr_df  = pd.merge(normal_pca_df,cdr_df.dropna(),left_index=True, right_index=True, how='left')

In [ ]:
normal_pca_cdr_df.head(10)

In [ ]:
nona_normal_pca_cdr_df = normal_pca_cdr_df.dropna()

In [ ]:
fig = plt.figure()
ax = fig.gca(projection="3d")

x,y,z = nona_normal_pca_cdr_df[0], nona_normal_pca_cdr_df[1],nona_normal_pca_cdr_df[2]

ax.scatter(x,y,z)

ax.set_xlabel("pca 1")
ax.set_ylabel("pca 2")
ax.set_zlabel("pca 3")
plt.suptitle("k mean clustering(pca)")
plt.show()

In [ ]:
plt.scatter(nona_normal_pca_cdr_df[0], nona_normal_pca_cdr_df[1], alpha=.3)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
inertias = []

# Creating 10 K-Mean models while varying the number of clusters (k)
for k in range(1,10):
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(nona_normal_pca_cdr_df.iloc[:,:3])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(range(1,10), inertias, '-p', color='gold')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')

plt.show()

In [ ]:
model = KMeans(n_clusters=2)
model.fit(normal_pca_df.iloc[:,:3])

labels = model.predict(nona_normal_pca_cdr_df.iloc[:,:3])

fig = plt.figure()
ax = fig.gca(projection="3d")

x,y,z = nona_normal_pca_cdr_df[0], nona_normal_pca_cdr_df[1],nona_normal_pca_cdr_df[2]

ax.scatter(x,y,z, c=labels)

ax.set_xlabel("pca 1")
ax.set_ylabel("pca 2")
ax.set_zlabel("pca 3")
plt.suptitle("k mean clustering(pca)")

plt.show()

In [ ]:

fig = plt.figure()
ax = fig.gca(projection="3d")

x,y,z = nona_normal_pca_cdr_df[0], nona_normal_pca_cdr_df[1],nona_normal_pca_cdr_df[2]
cdr = nona_normal_pca_cdr_df["CDR"]
cdr2 = cdr2 = [ 0 if i>0.2  else 1 for i in cdr]

ax.scatter(x,y,z, c=cdr2)

ax.set_xlabel("pca 1")

ax.set_ylabel("pca 2")
ax.set_zlabel("pca 3")
plt.suptitle("k mean clustering(pca)")
plt.show()

In [ ]:
cdr = nona_normal_pca_cdr_df["CDR"]
cdr2 = [ 0 if i>0.2  else 1 for i in cdr]

위 결과로 k means clustering은 cdr를 정상 비정상으로 판별을 하여 그리 좋은 정확도를 가지지 않음

다른 방법을 활용해야됨

# RANDOM FOREST

## mci

### 0123

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
import numpy as np

# age&f/m/education

age_fm_edu = data.iloc[:,1:4]
y_0123 = data.iloc[:,4]
xx = pd.merge(cerad_df,age_fm_edu,left_index=True, right_index=True, how='left')

all = pd.merge(xx,y_0123,left_index=True, right_index=True, how='left').dropna()

def randomforest0123(data,n):
  normal_normal = []
  dem_normal = []
  for i in range(n):
# test train 분리

    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data.iloc[:,-1], test_size=0.3)

    rf0123 = RandomForestClassifier(oob_score=True)
    rf0123.fit(X_train,y_train)

    predicted = rf0123.predict(X_test)
    accuracy = accuracy_score(y_test, predicted)
    cm = confusion_matrix(y_test, predicted, labels = [0,1,2,3])
    #print(cm)

    nn = cm[0][0]/cm.sum(axis=1)[0] ; 
    normal_normal.append(nn)
    #print("정상 - 정상:",nn)
   

    dn = (cm.sum(axis=0)[0]-cm[0][0])/(cm.sum()-cm.sum(axis=1)[0]) ; dem_normal.append(dn)
    #print("치매 - 정상:",dn)

    #print('\nClassification Report\n')
    #print(classification_report(y_test, predicted))
  #print(type(normal_normal))
  #print(type(dem_normal))
  df = pd.DataFrame(dem_normal,columns=["치매 -> 정상"])
  df["정상 to 정상"]=normal_normal
  #print(df)
  return df
cm0123 = randomforest0123(all,100)
cm0123.describe()

### 0&1

In [ ]:
# 치매는 다 1 로 바꾸기
all["y0123"] = all.iloc[:,-1]
y01 = []
for i in all["y0123"]:
  if i == 0:
    y01.append(i)
  else:
    y01.append(1)

def randomforest01(x,y,n=100):
  normal_normal = []
  dem_normal = []
  for i in range(n):
# test train 분리

    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

    rf0123 = RandomForestClassifier(oob_score=True)
    rf0123.fit(X_train,y_train)

    predicted = rf0123.predict(X_test)
    accuracy = accuracy_score(y_test, predicted)
    cm = confusion_matrix(y_test, predicted, labels = [0,1])
    nn = cm[0][0]/cm.sum(axis=1)[0] ; 
    normal_normal.append(nn)
    dn = cm[1][0] / cm.sum(axis=1)[1] 
    dem_normal.append(dn)
  df = pd.DataFrame(dem_normal,columns=["치매 -> 정상"])
  df["정상 to 정상"]=normal_normal
  return df
  
cm01 = randomforest01(all.iloc[:,:-2],y01)
cm01.describe()

## cdr

### 0, 0.5, 1

In [ ]:
xx = pd.merge(cerad_df,age_fm_edu,left_index=True, right_index=True, how='left')
yy = cdr_df.iloc[:,-3]*2
all_cdr = pd.merge(xx,yy,left_index=True, right_index=True, how='left').dropna()
all_cdr["CDR"]=all_cdr.CDR.astype(int)

def randomforest0051(data,n=10):
  normal_normal = []
  dem_normal = []
  for i in range(n):
# test train 분리

    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data.iloc[:,-1], test_size=0.3)

    rf0123 = RandomForestClassifier()
    rf0123.fit(X_train,y_train)

    predicted = rf0123.predict(X_test)
    accuracy = accuracy_score(y_test, predicted)
    cm = confusion_matrix(y_test, predicted, labels = [0, 0.5, 1])
    #print(cm)
    nn = cm[0][0]/cm.sum(axis=1)[0] 
    normal_normal.append(nn)
    print("정상 - 정상:",nn)
    dn = (cm.sum(axis=0)[0]-cm[0][0])/(cm.sum()-cm.sum(axis=1)[0]) ; dem_normal.append(dn)
    print("치매 - 정상:",dn)

  df = pd.DataFrame(dem_normal,columns=["치매 -> 정상"])
  df["정상 to 정상"]=normal_normal
  #print(df)
  return df

cm0051 = randomforest0051(all_cdr)
cm0051.describe()

#### confusion matrix 를 보면 0->0 정상: 정상은 다 갔고, 치매 --> 치매가 아님 이 1개 그외는 좋음

In [ ]:
fig = plt.figure()
ax = fig.gca(projection="3d")

x,y,z = nona_normal_pca_cdr_df[0], nona_normal_pca_cdr_df[1],nona_normal_pca_cdr_df[2]

ax.scatter(x,y,z, c=nona_normal_pca_cdr_df.iloc[:,-1])

ax.set_xlabel("pca 1")

ax.set_ylabel("pca 2")
ax.set_zlabel("pca 3")
plt.suptitle("k mean clustering(pca)")
plt.show()

In [ ]:
cdr_df.tail(10)

In [ ]:
data.tail(10)

In [ ]:
mri_df.head()

In [ ]:
%matplotlib inline   
import matplotlib.pyplot as plt 
import seaborn as sns  

plt.figure(figsize=(50,50))
sns.heatmap(data = mri_df.corr(), annot=False, linewidths=.5, cmap='Blues')

In [ ]:
mri_cdr = pd.merge(mri_df,cdr_df,left_index=True, right_index=True, how='left')
mri_cdr.head()
plt.figure(figsize=(50,50))
sns.heatmap(data = mri_cdr.corr(), annot=False, linewidths=.5, cmap='Blues')

# MRI


In [ ]:
total = pd.merge(mri_df,y_0123,left_index=True, right_index=True, how='left').dropna()
total.iloc[:,-1] = total.iloc[:,-1].astype(int)

## 차원축소 해보자

### kernal pca fuction

In [ ]:
from scipy.spatial.distance import pdist, squareform
from scipy import exp
from scipy.linalg import eigh
import numpy as np

def rbf_kernel_pca(X, gamma, n_components):
    # MxN 차원의 데이터셋에서 샘플 간의 유클리디안 거리의 제곱을 계산합니다.
    sq_dists = pdist(X, 'sqeuclidean')

    # 샘플 간의 거리를 정방 대칭 행렬로 변환합니다.
    mat_sq_dists = squareform(sq_dists)

    # 커널 행렬을 계산합니다.
    K = exp(-gamma * mat_sq_dists)

    # 커널 행렬을 중앙에 맞춥니다.
    N = K.shape[0]
    one_n = np.ones((N, N)) / N
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)

    # 중앙에 맞춰진 커널 행렬의 고윳값과 고유벡터를 구합니다.
    # scipy.linalg.eigh 함수는 오름차순으로 반환합니다.
    eigvals, eigvecs = eigh(K)
    eigvals, eigvecs = eigvals[::-1], eigvecs[:, ::-1]

    # 최상위 k 개의 고유벡터를 선택합니다(결과값은 투영된 샘플입니다).
    X_pc = np.column_stack([eigvecs[:, i]
                            for i in range(n_components)])

    return X_pc

In [ ]:
x = total.iloc[:,:-1]
X_kpca = rbf_kernel_pca(x,gamma=10,n_components=3)
print(X_kpca)

In [ ]:
from sklearn.decomposition import KernelPCA
x = total.iloc[:,:-1]
y = total.iloc[:,-1]

kpca = KernelPCA(n_components = 3, kernel="poly",gamma=10)
X_kpca = kpca.fit_transform(x)

kpca = pd.DataFrame(X_kpca)
kpca["y"] = y
kpca.columns=["pca1","pca2","pca3","y"]
kpca = kpca.dropna()
kpca.head(3)


In [ ]:
kpca.isna().sum()

In [ ]:
kpca["y"] = kpca["y"].astype(int)
d0 = kpca.query(" y == 0 ")
d1 = kpca.query(" y == 1 ")
d2 = kpca.query(" y == 2 ")
d3 = kpca.query(" y == 3 ")

In [ ]:
kpca.describe()

# 데이터 생성: smote etc..

In [ ]:
total_mri = mri_df
total_mri["y"]=y_0123
total_mri = total_mri.dropna()
total_mri.y = total_mri.y.astype(int)

total_mri.y.hist()

In [ ]:
d0 = total_mri.query(" y == 0 ")
d1 = total_mri.query(" y == 1 ")
d2 = total_mri.query(" y == 2 ")
d3 = total_mri.query(" y == 3 ")

In [ ]:
plt.figure(figsize=(50,50))
sns.heatmap(data = total_mri.corr(), annot=False, linewidths=.5, cmap='Blues')

In [ ]:
mri_cor = pd.DataFrame(total_mri.corr())
cor = mri_cor["y"][:-1]

plus4 = mri_cor.query("(y >= 0.4)")
minus4 =  mri_cor.query("(y <= -0.4)")
zero =  mri_cor.query("(y<0.1) & (y> -0.1)")

In [ ]:
plus4

In [ ]:
minus4

In [ ]:
zero

### smote: multi class classifier

https://machinelearningmastery.com/multi-class-imbalanced-classification/

In [ ]:
# load and summarize the dataset
from collections import Counter
from sklearn.preprocessing import LabelEncoder

x = total_mri.iloc[:,:-1]
y = total_mri.iloc[:,-1]

# label encode the target variable
y = LabelEncoder().fit_transform(y)
# summarize distribution
counter = Counter(y)
for k,v in counter.items():
	per = v / len(y) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE

x = total_mri.iloc[:,:-1]
y = total_mri.iloc[:,-1]

# transform the dataset & k_neighboors = 4 한 이유는 젤 작은 클라스 갯수가 5라서 k가 6 이상이면 못씀
oversample = SMOTE(k_neighbors=4)
x_, y_ = oversample.fit_resample(x, y)
# summarize distribution
counter = Counter(y_)
for k,v in counter.items():
	per = v / len(y) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

# anova 로 뽑은 파라미터로 random forest 해보기

In [ ]:
colname = [
 'CERAD_J1',
 'CERAD_J2',
 'CERAD_J3',
 'CERAD_J4',
 'CERAD_J6',
 'CERAD_J7',
 'CERAD_J8',
 'CERAD_J9A',
 'CERAD_J9B',
 'CERAD_J가1',
 'CERAD_J가2',
 'CDR',
 'Left-Inf-Lat-Vent',
 'Right-Inf-Lat-Vent',
 'lh_superiorparietal_volume',
 'rh_fusiform_volume',
 'rh_temporalpole_volume',
 'lh_caudalmiddlefrontal_thickness',
 'lh_fusiform_thickness',
 'lh_inferiortemporal_thickness',
 'lh_isthmuscingulate_thickness',
 'lh_lingual_thickness',
 'lh_middletemporal_thickness',
 'lh_parsopercularis_thickness',
 'lh_parstriangularis_thickness',
 'lh_postcentral_thickness',
 'lh_posteriorcingulate_thickness',
 'lh_precentral_thickness',
 'lh_precuneus_thickness',
 'lh_rostralmiddlefrontal_thickness',
 'lh_superiorfrontal_thickness',
 'lh_superiortemporal_thickness',
 'lh_supramarginal_thickness',
 'lh_temporalpole_thickness',
 'lh_MeanThickness_thickness',
 'rh_bankssts_thickness',
 'rh_caudalmiddlefrontal_thickness',
 'rh_entorhinal_thickness',
 'rh_fusiform_thickness',
 'rh_parsorbitalis_thickness',
 'rh_posteriorcingulate_thickness',
 'rh_rostralmiddlefrontal_thickness',
 'rh_superiorfrontal_thickness',
 'rh_superiortemporal_thickness',
 'rh_supramarginal_thickness',
 'rh_frontalpole_thickness',
 'rh_temporalpole_thickness',
 'rh_MeanThickness_thickness',
 'lh_medialorbitofrontal_meancurv',
 'rh_entorhinal_meancurv',
 'Frontal regions',
 'Anterior/posterior cingulate regions',
 'Lateral parietal regions',
 'Lateral temporal regions']

In [ ]:
df_anova = data.loc[:,colname]
df_anova["y"] = data.iloc[:,4]
df_anova = df_anova.dropna()
df_anova.shape

In [ ]:
df_anova.y.hist()

In [ ]:
from sklearn.metrics import plot_confusion_matrix
X = df_anova.iloc[:,:-1]
y = df_anova.iloc[:,-1]

from scipy.stats import rankdata

anova_col_see = pd.DataFrame(index=colname)
#anova_col_see.sort_values(by="importance", ascending=False)

def randomforest_anova(datax,datay,df,n=30):
  X= datax
  y = datay
  score_list = []
  
  for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    score = clf.score(X_test,y_test)
    predicted = clf.predict(X_test)
    score = clf.score(X_test,y_test)
    print(score)
    print('\nClassification Report\n')
    print(classification_report(y_test, predicted))
    plot_confusion_matrix(clf, X_test, y_test)  
    plt.show() 
    score_list.append(score)
    importance = clf.feature_importances_
    df[i] = rankdata(importance)
  return df,score_list

df,scorelist = randomforest_anova(X,y,anova_col_see)
anova_rank = pd.DataFrame(index=colname)
rank_sum = df.sum(axis=1)
anova_rank["sum"] = rank_sum
anova_rank["rank"] = rankdata(rank_sum)

mat_mul = np.matmul(df.to_numpy(),scorelist)
anova_rank["weighted importance"] = mat_mul
anova_rank["weighted importace rank"] = rankdata(mat_mul)
anova_rank.sort_values(by="weighted importace rank", ascending=False)

### anova - smote - random forest

In [ ]:
X = df_anova.iloc[:,:-1]
y = df_anova.iloc[:,-1]

def randomforest_anova_smote(datax,datay,df,n=30):
  X = datax
  y = datay
  score_list = []
  
  for i in range(n):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

    # transform the dataset & k_neighboors = 4 한 이유는 젤 작은 클라스 갯수가 5라서 k가 6 이상이면 못씀
    oversample = SMOTE(k_neighbors=1)
    X_train, y_train = oversample.fit_resample(X_train, y_train)
    X_train = pd.DataFrame(X_train,columns=colname)
    # summarize distribution
    counter = Counter(y_train)
    for k,v in counter.items():
      per = v / len(y) * 100
      print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
    # plot the distribution
    plt.bar(counter.keys(), counter.values())
    plt.show()


    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    score = clf.score(X_test,y_test)
    predicted = clf.predict(X_test)
    score = clf.score(X_test,y_test)
    print(score)
    print('\nClassification Report\n')
    print(classification_report(y_test, predicted))
    plot_confusion_matrix(clf, X_test, y_test)  
    plt.show() 
    score_list.append(score)
    importance = clf.feature_importances_
    df[i] = rankdata(importance)
  return df,score_list

anova_col_see_smote = pd.DataFrame(index=colname)
df_smote,scorelist_smote = randomforest_anova_smote(X,y,anova_col_see_smote)

anova_rank_smote = pd.DataFrame(index=colname)
rank_sum = df_smote.sum(axis=1)
anova_rank_smote["sum_smote"] = rank_sum
anova_rank_smote["rank_smote"] = rankdata(rank_sum)

mat_mul = np.matmul(df.to_numpy(),scorelist)
anova_rank_smote["weighted importance_smote"] = mat_mul
anova_rank_smote["weighted importace rank_smote"] = rankdata(mat_mul)
anova_rank_smote.sort_values(by="weighted importace rank_smote", ascending=False)

In [ ]:
np.mean(scorelist_smote)

In [ ]:
total_rank = pd.merge(anova_rank,anova_rank_smote,left_index=True,right_index=True,how="left" )
total_rank["weighted importace rank_smote"] == total_rank["weighted importace rank"]

In [ ]:
total_rank.head()

In [ ]:
forplot = total_rank[["weighted importance_smote","weighted importance"]].sort_values(by="weighted importance_smote",ascending=False)
forplot=forplot.drop( ['CERAD_J1','CERAD_J2','CERAD_J3','CERAD_J4','CERAD_J6','CERAD_J7','CERAD_J8','CERAD_J9A','CERAD_J9B','CERAD_J가1','CERAD_J가2', 'CDR'])
forplot

In [ ]:
forplot.describe()

# frmi

https://newkimchiman.tistory.com/27?category=588529

fmri deep leaning methods
https://www.frontiersin.org/articles/10.3389/fninf.2018.00023/full



# 참고자료 : 이미지

https://www.kaggle.com/polomarco/visualizatio-3d-nifti-dicom-matlab-nrrd-files